# Задание
Бизнесу хочется понять, сколько пользователей в фитмосте и как долго они остаются с нами. Это значит, что они хотят увидеть ретеншн пользователей по их покупкам.

У вас есть два датасета: 

#### users_payments: 
account_id INT — айдишник пользователя, purchase_date datetime — дата покупки, amount INT — сумма покупки. 

#### accounts: 
account_id INT — айдишник пользователя, status String — статус пользователя, project_name String — код проекта


---
#### Содержание:

1) Подготовка данных

2) PY

3) SQL

4) Вывод по Ретеншну

5) Мнение по значимости и расчету метрики ретеншн для бизнеса

---
### 1) Подготовка данных

In [1]:
import pandas as pd
import pyarrow.parquet as pq
import sqlite3

In [2]:
accounts = pq.read_table('~/accounts.parquet')
users_payments = pq.read_table('~/users_payments.parquet')

users_payments = users_payments.to_pandas()
accounts = accounts.to_pandas()

In [3]:
users_payments.head()

,account_id,purchase_date,amount
0,474724.0,2022-12-31 04:39:55,584.0
1,789265.0,2021-10-31 22:48:01,807.0
2,418333.0,2021-03-31 17:26:43,629.0
3,265787.0,2021-06-30 11:21:04,555.0
4,755766.0,2022-12-31 17:19:33,628.0


In [4]:
accounts.head()

,account_id,status,project_name
0,474724,active,fm
1,789265,active,fm
2,418333,active,fm
3,265787,active,fm
4,755766,active,fm


In [5]:
print(users_payments.info())
print(accounts.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5601516 entries, 0 to 5601515
Data columns (total 3 columns):
 #   Column         Dtype         
---  ------         -----         
 0   account_id     float64       
 1   purchase_date  datetime64[ns]
 2   amount         float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 128.2 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429400 entries, 0 to 429399
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   account_id    429400 non-null  int64 
 1   status        409400 non-null  object
 2   project_name  429400 non-null  object
dtypes: int64(1), object(2)
memory usage: 9.8+ MB
None


Типы данных отличаются от ТЗ, меняю на необходимые. 

In [6]:
float_columns = ['account_id', 'amount']
users_payments = users_payments.dropna(subset=float_columns)
users_payments[float_columns] = users_payments[float_columns].astype(int)

# Меняю дату на '%Y-%m-%d' для удобства расчета. 
users_payments['purchase_date'] = pd.to_datetime(users_payments['purchase_date'], format='%Y-%m-%d').dt.date


object_columns = ['status', 'project_name']
accounts[object_columns] = accounts[object_columns].astype(str)
# Обратил внимание на стринг-у, object не поменялся в str, 
# как я понял, это произошло из-за того, что в этих колонках данные уже записаны в формате str.

In [7]:
accounts.status.value_counts()

active     380911
nan         20000
double      16144
blocked     12345
Name: status, dtype: int64

У юзеров несколько статусов, принял решение брать в расчет только активных пользователей.

In [8]:
accounts.project_name.value_counts()  

fm      400828
fm       20000
kd        8572
Name: project_name, dtype: int64

Пользователи проекта fm записались с ошибкой, предполагаю, что есть пробел. Увидел количественное сходство со статусом nan, возможно есть взаимосвязь.

In [9]:
cross_table_accounts = pd.crosstab(accounts['status'], accounts['project_name'])
print(cross_table_accounts)

project_name      fm   fm      kd
status                           
active        355599  17729  7583
blocked        11484    576   285
double         15085    735   324
nan            18660    960   380


Создал кросс-таблицу, все таки nan и fm не коррелируют.

In [10]:
grouped_accounts = accounts.groupby(['status', 'project_name']).size().reset_index(name='count')
print(grouped_accounts)

     status project_name   count
0    active           fm  355599
1    active         fm     17729
2    active           kd    7583
3   blocked           fm   11484
4   blocked         fm       576
5   blocked           kd     285
6    double           fm   15085
7    double         fm       735
8    double           kd     324
9       nan           fm   18660
10      nan         fm       960
11      nan           kd     380


На всякий случай вывел в ином варианте записи ту же таблицу, стало понятно, что нужно избавиться от пробелов.

In [11]:
accounts['project_name'] = accounts['project_name'].str.replace(' ', '')

In [12]:
fm_accounts = accounts[accounts['project_name'] == 'fm']

In [13]:
missing_values_users_payments = users_payments.isnull().sum()
missing_values_fm_accounts = fm_accounts.isnull().sum()

print("Пропущенные значения в 'users_payments':\n", missing_values_users_payments)
print("Пропущенные значения в 'accounts' для fm:\n", missing_values_fm_accounts)

Пропущенные значения в 'users_payments':
 account_id       0
purchase_date    0
amount           0
dtype: int64
Пропущенные значения в 'accounts' для fm:
 account_id      0
status          0
project_name    0
dtype: int64


Пропуски отсутствуют, отлично.

In [14]:
merged_fm_acc_up = pd.merge(users_payments, fm_accounts, on='account_id')

In [15]:
merged_fm_acc_up.head()

,account_id,purchase_date,amount,status,project_name
0,474724,2022-12-31,584,active,fm
1,474724,2022-04-30,529,active,fm
2,474724,2021-03-31,722,active,fm
3,474724,2023-04-30,820,active,fm
4,474724,2021-05-31,775,active,fm


In [16]:
active_users = merged_fm_acc_up[merged_fm_acc_up['status'] == 'active']

In [17]:
active_users.head()

,account_id,purchase_date,amount,status,project_name
0,474724,2022-12-31,584,active,fm
1,474724,2022-04-30,529,active,fm
2,474724,2021-03-31,722,active,fm
3,474724,2023-04-30,820,active,fm
4,474724,2021-05-31,775,active,fm


Мержу дф-ы проекта fm и с активным статусом айдишников 

In [19]:
excel_file_path = 'cleared_data.xlsx'
active_users.to_excel(excel_file_path, index=False)

print(f'Таблица сохранена в файл {excel_file_path}')

ValueError: This sheet is too large! Your sheet size is: 4852147, 5 Max sheet size is: 1048576, 16384

На всякий случай выгружаю чистые данные. Но в моем юпитере стоят ограничения на размер дф-ов.


---
### 2) PY

In [20]:
active_users['account_id'].nunique()

373328

In [21]:
active_users['first_transaction_month'] = active_users['purchase_date'].astype('datetime64[M]')

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Определяю дату первой транзакции только по месяцу и году.

In [22]:
active_users.groupby(['first_transaction_month'])['account_id'].agg(['count', 'nunique']).reset_index()

,first_transaction_month,count,nunique
0,2021-01-01,134123,113035
1,2021-02-01,135345,113892
2,2021-03-01,134846,113450
3,2021-04-01,134936,113404
4,2021-05-01,134706,113441
5,2021-06-01,135053,113586
6,2021-07-01,135222,113701
7,2021-08-01,134576,113288
8,2021-09-01,134748,113644
9,2021-10-01,134089,112825


In [23]:
id_ftm = active_users.groupby('account_id')['first_transaction_month'].min().reset_index()

In [24]:
id_ftm.head()

,account_id,first_transaction_month
0,3,2021-01-01
1,7,2021-02-01
2,8,2021-04-01
3,10,2021-03-01
4,12,2021-07-01


Для каждого айдишника определяю первую дату активности

In [25]:
min_date = active_users.first_transaction_month.min()
max_date = active_users.first_transaction_month.max()

In [26]:
month = pd.DataFrame(pd.date_range(min_date, max_date, freq='MS'))
month.columns = ['month']
month.head()

,month
0,2021-01-01
1,2021-02-01
2,2021-03-01
3,2021-04-01
4,2021-05-01


Генерирую календарь от мин до макс даты первой активности.

In [27]:
id_ftm['key'] = 1
month['key'] = 1

In [28]:
monthly_data_dynamics = id_ftm.merge(month,on='key')[['account_id', 'first_transaction_month', 'month']]

In [29]:
monthly_data_dynamics = monthly_data_dynamics[monthly_data_dynamics['month'] >= monthly_data_dynamics['first_transaction_month']]

In [30]:
monthly_data_dynamics.head()

,account_id,first_transaction_month,month
0,3,2021-01-01,2021-01-01
1,3,2021-01-01,2021-02-01
2,3,2021-01-01,2021-03-01
3,3,2021-01-01,2021-04-01
4,3,2021-01-01,2021-05-01


Мержу календарь и дф id_ftm, чтобы отобразить последовательнность.

In [31]:
trans_month = active_users.groupby(['account_id', 'first_transaction_month'])['amount'].sum().reset_index()

In [32]:
trans_month.columns = ['account_id', 'month', 'amount']
trans_month.head()

,account_id,month,amount
0,3,2021-01-01,535
1,3,2021-02-01,514
2,3,2021-06-01,726
3,3,2021-09-01,803
4,3,2021-12-01,567


Для каждого пользователя считаю месяц-год и покупку.

In [33]:
monthly_activity = monthly_data_dynamics.merge(trans_month, on=['account_id', 'month'], how='left')
# склеиваю при помощи how='left', чтобы как раз и отобразить пропуски
monthly_activity['active'] = (monthly_activity['amount'] > 0)*1
monthly_activity.head()

,account_id,first_transaction_month,month,amount,active
0,3,2021-01-01,2021-01-01,535.0,1
1,3,2021-01-01,2021-02-01,514.0,1
2,3,2021-01-01,2021-03-01,NaN,0
3,3,2021-01-01,2021-04-01,NaN,0
4,3,2021-01-01,2021-05-01,NaN,0


Объединяю monthly_data_dynamics и trans_month, чтобы получить пропуски, когда айдишник не был активен. 

Так же добавляю столбец-определитель, по которому и буду считать ретеншн.

In [34]:
retention_py = monthly_activity.pivot_table(index='first_transaction_month', columns='month', values='active', aggfunc='mean')
retention_py

# вывожу расчитанный ретеншен при помощи пивотника с использованием функции mean

month,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,...,2023-03-01,2023-04-01,2023-05-01,2023-06-01,2023-07-01,2023-08-01,2023-09-01,2023-10-01,2023-11-01,2023-12-01
first_transaction_month,,,,,,,,,,,,,,,,,,,,,
2021-01-01,1.0,0.302225,0.300394,0.299819,0.299341,0.301057,0.299695,0.300075,0.300615,0.297704,...,0.302137,0.300022,0.299659,0.300473,0.300155,0.300942,0.300765,0.298713,0.298394,0.300385
2021-02-01,NaN,1.000000,0.300138,0.300891,0.300426,0.302546,0.304327,0.301518,0.300991,0.301681,...,0.302421,0.300527,0.300000,0.300665,0.299849,0.301505,0.300928,0.301881,0.301505,0.300251
2021-03-01,NaN,NaN,1.000000,0.304994,0.306308,0.301881,0.305768,0.301215,0.306830,0.301233,...,0.302439,0.306956,0.301395,0.300081,0.305264,0.305138,0.300171,0.303464,0.300351,0.304058
2021-04-01,NaN,NaN,NaN,1.000000,0.304663,0.306011,0.305363,0.302797,0.306193,0.302512,...,0.307307,0.306348,0.308370,0.301216,0.311533,0.303860,0.305234,0.304637,0.301086,0.301268
2021-05-01,NaN,NaN,NaN,NaN,1.000000,0.307675,0.303657,0.306150,0.306522,0.308531,...,0.306373,0.306485,0.305145,0.304959,0.310131,0.308196,0.301239,0.306559,0.302578,0.305108
2021-06-01,NaN,NaN,NaN,NaN,NaN,1.000000,0.301480,0.300242,0.310465,0.300457,...,0.298574,0.309389,0.304600,0.312026,0.309228,0.302663,0.301803,0.308690,0.304170,0.305354
2021-07-01,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.313680,0.303086,0.312298,...,0.303240,0.318517,0.304468,0.299632,0.304622,0.303624,0.310533,0.314371,0.311300,0.309995
2021-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.305457,0.305679,...,0.310781,0.314441,0.314996,0.318101,0.313886,0.310115,0.315217,0.311446,0.308341,0.314330
2021-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.312767,...,0.323105,0.300295,0.303905,0.318018,0.308664,0.315720,0.323597,0.298326,0.312110,0.317197


In [35]:
style_py = (retention_py
            .style
            .set_caption('User retention by cohort')
            .background_gradient(cmap='viridis')
            .highlight_null('white')
            .format("{:.2%}", na_rep=""))
style_py

# делаю более приятное оформление

month,2021-01-01 00:00:00,2021-02-01 00:00:00,2021-03-01 00:00:00,2021-04-01 00:00:00,2021-05-01 00:00:00,2021-06-01 00:00:00,2021-07-01 00:00:00,2021-08-01 00:00:00,2021-09-01 00:00:00,2021-10-01 00:00:00,2021-11-01 00:00:00,2021-12-01 00:00:00,2022-01-01 00:00:00,2022-02-01 00:00:00,2022-03-01 00:00:00,2022-04-01 00:00:00,2022-05-01 00:00:00,2022-06-01 00:00:00,2022-07-01 00:00:00,2022-08-01 00:00:00,2022-09-01 00:00:00,2022-10-01 00:00:00,2022-11-01 00:00:00,2022-12-01 00:00:00,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00:00,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00,2023-07-01 00:00:00,2023-08-01 00:00:00,2023-09-01 00:00:00,2023-10-01 00:00:00,2023-11-01 00:00:00,2023-12-01 00:00:00
first_transaction_month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00,100.00%,30.22%,30.04%,29.98%,29.93%,30.11%,29.97%,30.01%,30.06%,29.77%,30.08%,29.89%,30.21%,29.91%,30.27%,30.04%,29.88%,30.11%,29.97%,29.95%,30.22%,30.00%,29.84%,29.90%,30.05%,29.89%,30.21%,30.00%,29.97%,30.05%,30.02%,30.09%,30.08%,29.87%,29.84%,30.04%
2021-02-01 00:00:00,,100.00%,30.01%,30.09%,30.04%,30.25%,30.43%,30.15%,30.10%,30.17%,30.02%,30.43%,30.28%,29.99%,30.18%,30.09%,30.43%,30.10%,30.34%,30.24%,30.43%,30.23%,29.76%,30.25%,30.40%,30.30%,30.24%,30.05%,30.00%,30.07%,29.98%,30.15%,30.09%,30.19%,30.15%,30.03%
2021-03-01 00:00:00,,,100.00%,30.50%,30.63%,30.19%,30.58%,30.12%,30.68%,30.12%,30.39%,30.28%,30.28%,30.42%,30.33%,30.03%,30.58%,30.26%,30.42%,30.99%,30.26%,30.41%,30.41%,30.49%,30.17%,30.10%,30.24%,30.70%,30.14%,30.01%,30.53%,30.51%,30.02%,30.35%,30.04%,30.41%
2021-04-01 00:00:00,,,,100.00%,30.47%,30.60%,30.54%,30.28%,30.62%,30.25%,30.05%,30.82%,30.72%,30.45%,30.20%,30.40%,30.77%,30.75%,30.56%,30.90%,30.80%,30.42%,30.26%,30.67%,30.43%,30.71%,30.73%,30.63%,30.84%,30.12%,31.15%,30.39%,30.52%,30.46%,30.11%,30.13%
2021-05-01 00:00:00,,,,,100.00%,30.77%,30.37%,30.61%,30.65%,30.85%,30.73%,30.03%,30.57%,30.77%,30.34%,30.52%,30.38%,31.21%,30.74%,30.30%,31.01%,30.52%,30.59%,30.87%,30.84%,30.83%,30.64%,30.65%,30.51%,30.50%,31.01%,30.82%,30.12%,30.66%,30.26%,30.51%
2021-06-01 00:00:00,,,,,,100.00%,30.15%,30.02%,31.05%,30.05%,30.02%,30.67%,30.62%,31.03%,31.13%,30.91%,30.93%,31.08%,31.29%,30.69%,30.44%,30.40%,31.36%,30.27%,30.51%,30.94%,29.86%,30.94%,30.46%,31.20%,30.92%,30.27%,30.18%,30.87%,30.42%,30.54%
2021-07-01 00:00:00,,,,,,,100.00%,31.37%,30.31%,31.23%,29.92%,30.82%,30.57%,31.38%,29.50%,31.61%,31.09%,31.27%,30.92%,30.24%,31.15%,30.38%,31.04%,30.66%,30.52%,30.55%,30.32%,31.85%,30.45%,29.96%,30.46%,30.36%,31.05%,31.44%,31.13%,31.00%
2021-08-01 00:00:00,,,,,,,,100.00%,30.55%,30.57%,30.83%,31.28%,31.52%,31.62%,31.08%,30.17%,30.28%,31.51%,31.40%,30.70%,30.71%,31.72%,31.99%,31.82%,31.06%,30.28%,31.08%,31.44%,31.50%,31.81%,31.39%,31.01%,31.52%,31.14%,30.83%,31.43%
2021-09-01 00:00:00,,,,,,,,,100.00%,31.28%,30.80%,31.75%,32.29%,30.16%,31.33%,32.51%,32.11%,31.82%,29.91%,31.26%,31.05%,30.57%,31.60%,31.88%,30.21%,30.69%,32.31%,30.03%,30.39%,31.80%,30.87%,31.57%,32.36%,29.83%,31.21%,31.72%


In [36]:
excel_file_path = 'retention_py.xlsx'
retention_py.to_excel(excel_file_path, index=False)

print(f'Таблица сохранена в файл {excel_file_path}')

Таблица сохранена в файл retention_py.xlsx


В конце концов сейвлю расчет.

In [37]:
monthly_activity[monthly_activity['first_transaction_month'] == '2023-03-01']

,account_id,first_transaction_month,month,amount,active
1002140,79899,2023-03-01,2023-03-01,1694.0,1
1002141,79899,2023-03-01,2023-04-01,865.0,1
1002142,79899,2023-03-01,2023-05-01,642.0,1
1002143,79899,2023-03-01,2023-06-01,1053.0,1
1002144,79899,2023-03-01,2023-07-01,1404.0,1
1002145,79899,2023-03-01,2023-08-01,NaN,0
1002146,79899,2023-03-01,2023-09-01,536.0,1
1002147,79899,2023-03-01,2023-10-01,NaN,0
1002148,79899,2023-03-01,2023-11-01,NaN,0
1002149,79899,2023-03-01,2023-12-01,NaN,0


Ретеншн за последние месяцы первой активности пользователей показались мне странными, но когда я вывел дебаг, все встало на свои места - пользователей с первой активностью в последние месяцы временного промежутка очень мало, тем самым и скачет процент.

---
### 3) SQL

In [38]:
con = sqlite3.connect('database.db')
active_users.to_sql('cleared_data', con, index=False, if_exists='replace')

In [39]:
sql = 'select * from cleared_data limit 50'
result = pd.read_sql(sql, con)
result.head()

,account_id,purchase_date,amount,status,project_name,first_transaction_month
0,474724,2022-12-31,584,active,fm,2022-12-01 00:00:00
1,474724,2022-04-30,529,active,fm,2022-04-01 00:00:00
2,474724,2021-03-31,722,active,fm,2021-03-01 00:00:00
3,474724,2023-04-30,820,active,fm,2023-04-01 00:00:00
4,474724,2021-05-31,775,active,fm,2021-05-01 00:00:00


Загружаю подготовленный дф в базу и отображаю, чтобы убедиться, что все пошло по плану.

In [40]:
min = '''select date(min(purchase_date), 'start of month') from cleared_data '''

min_data = pd.read_sql(min, con)
min_data

,"date(min(purchase_date), 'start of month')"
0,2021-01-01


In [41]:
max = '''select date(max(purchase_date), 'start of month') from cleared_data '''

max_data = pd.read_sql(max, con)
max_data

,"date(max(purchase_date), 'start of month')"
0,2023-12-01


Расчитал промежутки (мин макс)

In [42]:
months = f'''
    WITH RECURSIVE dates(date) AS (
        VALUES (({min}))
        UNION ALL
        SELECT DATE(date, '+1 month')
        FROM dates
        WHERE date < ({max})
    )
    SELECT date FROM dates
    LIMIT 5
'''

months_data = pd.read_sql(months, con)

months_data.head()

,date
0,2021-01-01
1,2021-02-01
2,2021-03-01
3,2021-04-01
4,2021-05-01


Написал и проверил рекурсивный запрос для генерации последовательности даты в нужном промежутке.

Ниже основной код:

In [43]:
sql = f'''

-- Временная таблица id_ftm, с первой транзакцией для каждого айдишника
WITH id_ftm AS (
        SELECT 
               -- Формирую выборку account_id и первой даты транзакции, округленной до начала месяца
               account_id, 
               DATE(MIN(purchase_date), 'start of month') AS first_transaction_month
        FROM 
               cleared_data
        GROUP BY 
               account_id
    ),

    -- Временная таблица dates с рекурсивным запросом календаря
    dates AS (
        WITH RECURSIVE dates(date) AS (
            VALUES (({min}))
            UNION ALL
            SELECT DATE(date, '+1 month')
            FROM dates
            WHERE date < ({max})
        )
        SELECT date FROM dates
    ),
    
    -- Временная таблица monthly_data_dynamics с объединением id_ftm и dates
    monthly_data_dynamics AS (
        SELECT 
               ftm.account_id, 
               ftm.first_transaction_month, 
               dts.date AS month 
        FROM 
               id_ftm ftm
        JOIN dates dts ON dts.date >= ftm.first_transaction_month
    ),

    -- Временная таблица trans_month с суммами транзакций для каждого месяца и аккаунта
    trans_month AS (
        SELECT 
            account_id, 
            DATE((purchase_date), 'start of month') AS month,
            SUM(amount) AS amount_sum
        FROM cleared_data
        GROUP BY account_id, 
                 DATE((purchase_date), 'start of month')
    ),

    -- Временная таблица monthly_activity объединяющая данные monthly_data_dynamics и trans_month
    monthly_activity AS (
        SELECT 
               mdd.*, 
               tm.amount_sum,
               -- Написал флаг active 0 или 1
               CASE WHEN tm.amount_sum > 0 THEN 1 ELSE 0 END AS active
        FROM 
               monthly_data_dynamics mdd
        LEFT JOIN trans_month tm ON mdd.account_id = tm.account_id AND mdd.month = tm.month
    )

-- Окончательный запрос, выводящий ретеншн в процентах, округленный до сотых
SELECT 
       first_transaction_month, 
       month, 
       ROUND(AVG(active) * 100, 2) AS retention
FROM 
       monthly_activity 
GROUP BY 
       first_transaction_month, 
       month

'''


retention_sql = pd.read_sql(sql, con)
retention_sql

,first_transaction_month,month,retention
0,2021-01-01,2021-01-01,100.00
1,2021-01-01,2021-02-01,30.22
2,2021-01-01,2021-03-01,30.04
3,2021-01-01,2021-04-01,29.98
4,2021-01-01,2021-05-01,29.93
...,...,...,...
616,2023-03-01,2023-08-01,50.00
617,2023-03-01,2023-09-01,100.00
618,2023-03-01,2023-10-01,50.00
619,2023-03-01,2023-11-01,0.00


Далее пишу пивотник для красивого отображения результата.

In [44]:
retention_sql = retention_sql.pivot_table(index='first_transaction_month', columns='month', values='retention')
retention_sql

month,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,...,2023-03-01,2023-04-01,2023-05-01,2023-06-01,2023-07-01,2023-08-01,2023-09-01,2023-10-01,2023-11-01,2023-12-01
first_transaction_month,,,,,,,,,,,,,,,,,,,,,
2021-01-01,100.0,30.22,30.04,29.98,29.93,30.11,29.97,30.01,30.06,29.77,...,30.21,30.00,29.97,30.05,30.02,30.09,30.08,29.87,29.84,30.04
2021-02-01,NaN,100.00,30.01,30.09,30.04,30.25,30.43,30.15,30.10,30.17,...,30.24,30.05,30.00,30.07,29.98,30.15,30.09,30.19,30.15,30.03
2021-03-01,NaN,NaN,100.00,30.50,30.63,30.19,30.58,30.12,30.68,30.12,...,30.24,30.70,30.14,30.01,30.53,30.51,30.02,30.35,30.04,30.41
2021-04-01,NaN,NaN,NaN,100.00,30.47,30.60,30.54,30.28,30.62,30.25,...,30.73,30.63,30.84,30.12,31.15,30.39,30.52,30.46,30.11,30.13
2021-05-01,NaN,NaN,NaN,NaN,100.00,30.77,30.37,30.61,30.65,30.85,...,30.64,30.65,30.51,30.50,31.01,30.82,30.12,30.66,30.26,30.51
2021-06-01,NaN,NaN,NaN,NaN,NaN,100.00,30.15,30.02,31.05,30.05,...,29.86,30.94,30.46,31.20,30.92,30.27,30.18,30.87,30.42,30.54
2021-07-01,NaN,NaN,NaN,NaN,NaN,NaN,100.00,31.37,30.31,31.23,...,30.32,31.85,30.45,29.96,30.46,30.36,31.05,31.44,31.13,31.00
2021-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00,30.55,30.57,...,31.08,31.44,31.50,31.81,31.39,31.01,31.52,31.14,30.83,31.43
2021-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00,31.28,...,32.31,30.03,30.39,31.80,30.87,31.57,32.36,29.83,31.21,31.72


In [45]:
style_sql = (retention_sql
            .style
            .set_caption('User retention by cohort')
            .background_gradient(cmap='viridis')
            .highlight_null('white')
            .format("{:}%", na_rep=""))
style_sql

month,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01,2023-01-01,2023-02-01,2023-03-01,2023-04-01,2023-05-01,2023-06-01,2023-07-01,2023-08-01,2023-09-01,2023-10-01,2023-11-01,2023-12-01
first_transaction_month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-01-01,100.0%,30.22%,30.04%,29.98%,29.93%,30.11%,29.97%,30.01%,30.06%,29.77%,30.08%,29.89%,30.21%,29.91%,30.27%,30.04%,29.88%,30.11%,29.97%,29.95%,30.22%,30.0%,29.84%,29.9%,30.05%,29.89%,30.21%,30.0%,29.97%,30.05%,30.02%,30.09%,30.08%,29.87%,29.84%,30.04%
2021-02-01,,100.0%,30.01%,30.09%,30.04%,30.25%,30.43%,30.15%,30.1%,30.17%,30.02%,30.43%,30.28%,29.99%,30.18%,30.09%,30.43%,30.1%,30.34%,30.24%,30.43%,30.23%,29.76%,30.25%,30.4%,30.3%,30.24%,30.05%,30.0%,30.07%,29.98%,30.15%,30.09%,30.19%,30.15%,30.03%
2021-03-01,,,100.0%,30.5%,30.63%,30.19%,30.58%,30.12%,30.68%,30.12%,30.39%,30.28%,30.28%,30.42%,30.33%,30.03%,30.58%,30.26%,30.42%,30.99%,30.26%,30.41%,30.41%,30.49%,30.17%,30.1%,30.24%,30.7%,30.14%,30.01%,30.53%,30.51%,30.02%,30.35%,30.04%,30.41%
2021-04-01,,,,100.0%,30.47%,30.6%,30.54%,30.28%,30.62%,30.25%,30.05%,30.82%,30.72%,30.45%,30.2%,30.4%,30.77%,30.75%,30.56%,30.9%,30.8%,30.42%,30.26%,30.67%,30.43%,30.71%,30.73%,30.63%,30.84%,30.12%,31.15%,30.39%,30.52%,30.46%,30.11%,30.13%
2021-05-01,,,,,100.0%,30.77%,30.37%,30.61%,30.65%,30.85%,30.73%,30.03%,30.57%,30.77%,30.34%,30.52%,30.38%,31.21%,30.74%,30.3%,31.01%,30.52%,30.59%,30.87%,30.84%,30.83%,30.64%,30.65%,30.51%,30.5%,31.01%,30.82%,30.12%,30.66%,30.26%,30.51%
2021-06-01,,,,,,100.0%,30.15%,30.02%,31.05%,30.05%,30.02%,30.67%,30.62%,31.03%,31.13%,30.91%,30.93%,31.08%,31.29%,30.69%,30.44%,30.4%,31.36%,30.27%,30.51%,30.94%,29.86%,30.94%,30.46%,31.2%,30.92%,30.27%,30.18%,30.87%,30.42%,30.54%
2021-07-01,,,,,,,100.0%,31.37%,30.31%,31.23%,29.92%,30.82%,30.57%,31.38%,29.5%,31.61%,31.09%,31.27%,30.92%,30.24%,31.15%,30.38%,31.04%,30.66%,30.52%,30.55%,30.32%,31.85%,30.45%,29.96%,30.46%,30.36%,31.05%,31.44%,31.13%,31.0%
2021-08-01,,,,,,,,100.0%,30.55%,30.57%,30.83%,31.28%,31.52%,31.62%,31.08%,30.17%,30.28%,31.51%,31.4%,30.7%,30.71%,31.72%,31.99%,31.82%,31.06%,30.28%,31.08%,31.44%,31.5%,31.81%,31.39%,31.01%,31.52%,31.14%,30.83%,31.43%
2021-09-01,,,,,,,,,100.0%,31.28%,30.8%,31.75%,32.29%,30.16%,31.33%,32.51%,32.11%,31.82%,29.91%,31.26%,31.05%,30.57%,31.6%,31.88%,30.21%,30.69%,32.31%,30.03%,30.39%,31.8%,30.87%,31.57%,32.36%,29.83%,31.21%,31.72%


In [46]:
excel_file_path = 'retention_sql.xlsx'
retention_sql.to_excel(excel_file_path, index=False)

print(f'Таблица сохранена в файл {excel_file_path}')

Таблица сохранена в файл retention_sql.xlsx


---
### 4) Вывод по Ретеншну по Когортам:

1. Ретеншн пользователей в целом демонстрирует постепенное снижение с течением времени для большинства когорт. Это типичный паттерн для многих отраслей.

2. Первые когорты (например, январь 2021 года) имеют более высокие показатели ретеншна в первые месяцы по сравнению с поздними когортами. Некоторые когорты имеют отклонения в показателях ретеншна, в то время как у других наблюдается более стабильное снижение. Несколько когорт поддерживают стабильный показатель ретеншна даже после нескольких месяцев => устойчивая база пользователей.

3. Есть и аномалии, например, резкие снижения или повышения показателей ретеншна, они могут быть обусловлены внешними факторами, изменениями в продукте/услуге или действиями отдела маркетинга.

4. Для улучшения общего ретеншна лучше применить более глубокий анализ поведения пользователей, их предпочтений и возможных проблем. Например, индивидуализированные маркетинговые стратегии вовлечения, основанные на наблюдаемых паттернах в когортах с высоким ретеншном.

### 5) Мнение по значимости и расчету метрики ретеншн для бизнеса:

Зачем важно считать ретеншн?

1. Ретеншн - это ключевой показатель, отражающий то, насколько успешно бизнес удерживает клиентов с течением времени. Высокий ретеншн свидетельствует о долгосрочной лояльности клиентов, что может быть критически важным для устойчивого развития.

2. Расходы на привлечение новых клиентов могут быть значительными, в то время как удержание существующих может сэкономить средства.

3. Ретеншн также служит индикатором удовлетворенности клиентов. Высокие показатели ретеншна свидетельствуют о качественных продуктах или услугах, что может быть важным конкурентным преимуществом.

4. Стабильный ретеншн облегчает прогнозирование будущих поступлений и тем самым позволяет бизнесу более точно планировать свои стратегии и расходы.
---
Нюансы при расчете ретеншна:

1. Необходимо четко определить временной интервал для расчета ретеншна.

2. Каждый бизнес уникален, и метрика ретеншна может иметь разную значимость в зависимости от отрасли и типа продукта/услуги. Важно учитывать специфику бизнеса при интерпретации показателей.

3. При расчёте важно учитывать другие метрики и факторы, например, маркетинговые инициативы, обновления продукта и конкурентная обстановка.

4. В некоторых отраслях ретеншн может подвергаться влиянию сезонных колебаний, это также нужно учитывать.


---
Заключение:

Ретеншн представляет собой важную метрику для бизнеса, и игнорирование ее может привести к потере ценных клиентов и упущенным возможностям для экономии ресурсов. Этот показатель не только отражает текущую устойчивость, но и отражает возможности потенциального роста. Необходимо учитывать контекст и особенности своего бизнеса при интерпретации результатов и принятии решений при анализе ретеншна.




